In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("My Spark App") \
    .getOrCreate()

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import io

In [3]:
# Load the dataset

from google.colab import files
uploaded = files.upload()

Saving processed_crypto_data.csv to processed_crypto_data.csv


In [4]:
 # Read the file into a pandas DataFrame
df = pd.read_csv("processed_crypto_data.csv")

In [5]:
#Adding a Spread Column
# to calculate the price spread for each pair

df['Spread'] = df['High'] - df['Low']

In [6]:
print(df.head())

        Pair   High    Low  Last Price        Volume  \
0  Chainlink  22.15  20.11       21.76  2.528254e+05   
1    Cardano   1.09   0.99        1.04  1.981526e+07   
2    Polygon   0.49   0.45        0.48  9.776040e+05   
3   Dogecoin   0.39   0.35        0.38  9.675374e+07   
4     Ripple   3.20   2.68        3.05  1.274103e+08   

                       Timestamp        Date  Spread  
0  2025-01-16T00:21:20.000-05:00  2025-01-16    2.04  
1  2025-01-16T00:38:24.000-05:00  2025-01-16    0.10  
2  2025-01-16T00:56:28.000-05:00  2025-01-16    0.04  
3  2025-01-16T01:02:29.000-05:00  2025-01-16    0.04  
4  2025-01-16T01:05:29.000-05:00  2025-01-16    0.52  


In [7]:
#Calculating the position of the Last Price within the range
df['Position'] = (df['Last Price'] - df['Low']) / (df['High'] - df['Low'])


In [8]:
print(df.head())

        Pair   High    Low  Last Price        Volume  \
0  Chainlink  22.15  20.11       21.76  2.528254e+05   
1    Cardano   1.09   0.99        1.04  1.981526e+07   
2    Polygon   0.49   0.45        0.48  9.776040e+05   
3   Dogecoin   0.39   0.35        0.38  9.675374e+07   
4     Ripple   3.20   2.68        3.05  1.274103e+08   

                       Timestamp        Date  Spread  Position  
0  2025-01-16T00:21:20.000-05:00  2025-01-16    2.04  0.808824  
1  2025-01-16T00:38:24.000-05:00  2025-01-16    0.10  0.500000  
2  2025-01-16T00:56:28.000-05:00  2025-01-16    0.04  0.750000  
3  2025-01-16T01:02:29.000-05:00  2025-01-16    0.04  0.750000  
4  2025-01-16T01:05:29.000-05:00  2025-01-16    0.52  0.711538  


In [9]:
#To Compute the percentage volatility of each cryptocurrency

df['Volatility'] = ((df['High'] - df['Low']) / df['Low']) * 100


In [10]:
print(df.head())

        Pair   High    Low  Last Price        Volume  \
0  Chainlink  22.15  20.11       21.76  2.528254e+05   
1    Cardano   1.09   0.99        1.04  1.981526e+07   
2    Polygon   0.49   0.45        0.48  9.776040e+05   
3   Dogecoin   0.39   0.35        0.38  9.675374e+07   
4     Ripple   3.20   2.68        3.05  1.274103e+08   

                       Timestamp        Date  Spread  Position  Volatility  
0  2025-01-16T00:21:20.000-05:00  2025-01-16    2.04  0.808824   10.144207  
1  2025-01-16T00:38:24.000-05:00  2025-01-16    0.10  0.500000   10.101010  
2  2025-01-16T00:56:28.000-05:00  2025-01-16    0.04  0.750000    8.888889  
3  2025-01-16T01:02:29.000-05:00  2025-01-16    0.04  0.750000   11.428571  
4  2025-01-16T01:05:29.000-05:00  2025-01-16    0.52  0.711538   19.402985  


In [11]:
#Extracting features like hour and minute from the Timestamp column

# Convert 'Timestamp' column to datetime objects
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

df['Hour'] = df['Timestamp'].dt.hour
df['Minute'] = df['Timestamp'].dt.minute

print(df.head())

        Pair   High    Low  Last Price        Volume  \
0  Chainlink  22.15  20.11       21.76  2.528254e+05   
1    Cardano   1.09   0.99        1.04  1.981526e+07   
2    Polygon   0.49   0.45        0.48  9.776040e+05   
3   Dogecoin   0.39   0.35        0.38  9.675374e+07   
4     Ripple   3.20   2.68        3.05  1.274103e+08   

                  Timestamp        Date  Spread  Position  Volatility  Hour  \
0 2025-01-16 00:21:20-05:00  2025-01-16    2.04  0.808824   10.144207     0   
1 2025-01-16 00:38:24-05:00  2025-01-16    0.10  0.500000   10.101010     0   
2 2025-01-16 00:56:28-05:00  2025-01-16    0.04  0.750000    8.888889     0   
3 2025-01-16 01:02:29-05:00  2025-01-16    0.04  0.750000   11.428571     1   
4 2025-01-16 01:05:29-05:00  2025-01-16    0.52  0.711538   19.402985     1   

   Minute  
0      21  
1      38  
2      56  
3       2  
4       5  


In [12]:
#Computing rolling averages for prices or volume over a time window:

df['Rolling_Avg_Volume'] = df['Volume'].rolling(window=3).mean()

print(df.head())

        Pair   High    Low  Last Price        Volume  \
0  Chainlink  22.15  20.11       21.76  2.528254e+05   
1    Cardano   1.09   0.99        1.04  1.981526e+07   
2    Polygon   0.49   0.45        0.48  9.776040e+05   
3   Dogecoin   0.39   0.35        0.38  9.675374e+07   
4     Ripple   3.20   2.68        3.05  1.274103e+08   

                  Timestamp        Date  Spread  Position  Volatility  Hour  \
0 2025-01-16 00:21:20-05:00  2025-01-16    2.04  0.808824   10.144207     0   
1 2025-01-16 00:38:24-05:00  2025-01-16    0.10  0.500000   10.101010     0   
2 2025-01-16 00:56:28-05:00  2025-01-16    0.04  0.750000    8.888889     0   
3 2025-01-16 01:02:29-05:00  2025-01-16    0.04  0.750000   11.428571     1   
4 2025-01-16 01:05:29-05:00  2025-01-16    0.52  0.711538   19.402985     1   

   Minute  Rolling_Avg_Volume  
0      21                 NaN  
1      38                 NaN  
2      56        7.015231e+06  
3       2        3.918220e+07  
4       5        7.504721e+0

In [13]:
#Apply log transformation to the Volume column to reduce skewness

df['Log_Volume'] = np.log1p(df['Volume'])

print(df.head())

        Pair   High    Low  Last Price        Volume  \
0  Chainlink  22.15  20.11       21.76  2.528254e+05   
1    Cardano   1.09   0.99        1.04  1.981526e+07   
2    Polygon   0.49   0.45        0.48  9.776040e+05   
3   Dogecoin   0.39   0.35        0.38  9.675374e+07   
4     Ripple   3.20   2.68        3.05  1.274103e+08   

                  Timestamp        Date  Spread  Position  Volatility  Hour  \
0 2025-01-16 00:21:20-05:00  2025-01-16    2.04  0.808824   10.144207     0   
1 2025-01-16 00:38:24-05:00  2025-01-16    0.10  0.500000   10.101010     0   
2 2025-01-16 00:56:28-05:00  2025-01-16    0.04  0.750000    8.888889     0   
3 2025-01-16 01:02:29-05:00  2025-01-16    0.04  0.750000   11.428571     1   
4 2025-01-16 01:05:29-05:00  2025-01-16    0.52  0.711538   19.402985     1   

   Minute  Rolling_Avg_Volume  Log_Volume  
0      21                 NaN   12.440458  
1      38                 NaN   16.801963  
2      56        7.015231e+06   13.792861  
3       2   

In [14]:
#Checking for missing values

print(df.isnull().sum())

Pair                  0
High                  0
Low                   0
Last Price            0
Volume                0
Timestamp             0
Date                  0
Spread                0
Position              0
Volatility            0
Hour                  0
Minute                0
Rolling_Avg_Volume    2
Log_Volume            0
dtype: int64


In [15]:
#dropping the coloumns with NaN values
df = df.dropna(axis=1)

In [16]:
print(df.isnull().sum())

Pair          0
High          0
Low           0
Last Price    0
Volume        0
Timestamp     0
Date          0
Spread        0
Position      0
Volatility    0
Hour          0
Minute        0
Log_Volume    0
dtype: int64


In [17]:
# Save to CSV
df.to_csv('modified_crypto_data.csv', index=False)

In [18]:
from google.colab import files

files.download('modified_crypto_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>